In [ ]:
import json
with open('locomo10.json', 'r') as file:
    data = json.load(file)
    print(type(data))
    print(len(data))
    print(type(data[0]))
    print(data[0].keys())
    for key, value in data[0].items():
        print(f"{key}: {type(value)}")
    print(type(data[0]['qa'][0]))
    print(data[0]['qa'][0])
    # for key, value in data[0]['conversation'].items():
    #     print(f"{key}: {type(value)}")

<class 'list'>
10
<class 'dict'>
dict_keys(['qa', 'conversation', 'event_summary', 'observation', 'session_summary', 'sample_id'])
qa: <class 'list'>
conversation: <class 'dict'>
event_summary: <class 'dict'>
observation: <class 'dict'>
session_summary: <class 'dict'>
sample_id: <class 'str'>
<class 'dict'>
{'question': 'When did Caroline go to the LGBTQ support group?', 'answer': '7 May 2023', 'evidence': ['D1:3'], 'category': 2}
speaker_a: <class 'str'>
speaker_b: <class 'str'>
session_1_date_time: <class 'str'>
session_1: <class 'list'>
session_2_date_time: <class 'str'>
session_2: <class 'list'>
session_3_date_time: <class 'str'>
session_3: <class 'list'>
session_4_date_time: <class 'str'>
session_4: <class 'list'>
session_5_date_time: <class 'str'>
session_5: <class 'list'>
session_6_date_time: <class 'str'>
session_6: <class 'list'>
session_7_date_time: <class 'str'>
session_7: <class 'list'>
session_8_date_time: <class 'str'>
session_8: <class 'list'>
session_9_date_time: <class

In [6]:
with open('/root/nfs/hmj/proj/mem0/logs/lgm.log', 'r') as log_file:
    lines = log_file.read()
# for line in lines:
sum= 0
for line in lines.splitlines():
    d = line[-2:]
    sum+= int(d)
print(sum)

15911


In [2]:
import os

dpath_dir = '/root/nfs/hmj/proj/mem0/evaluation/src/dataset/all_conversations'
output_dir = '/root/nfs/hmj/proj/mem0/evaluation/src/dataset/all_conversations'

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 获取所有子文件夹，并排序（保证稳定对应 0-9）
subfolders = sorted(
    [f for f in os.listdir(dpath_dir) if os.path.isdir(os.path.join(dpath_dir, f))]
)

# 检查数量是否为 10
assert len(subfolders) == 10, f"期望 10 个子文件夹，但找到 {len(subfolders)} 个。"

# 遍历每一个子文件夹
for idx, folder in enumerate(subfolders):
    folder_path = os.path.join(dpath_dir, folder)
    output_file = os.path.join(output_dir, f"{idx}.txt")

    with open(output_file, "w", encoding="utf-8") as out_f:
        # 获取子文件夹下的 txt 文件并排序
        files = sorted([f for f in os.listdir(folder_path) if f.endswith(".txt")])

        for fname in files:
            file_path = os.path.join(folder_path, fname)

            # 读取原文件内容
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            # 写入合并文件
            out_f.write(content + "\n")

    print(f"[OK] 合并完成：{folder} → {idx}.txt")

print("全部完成！输出位置：", output_dir)


[OK] 合并完成：0_Caroline_Melanie_talk → 0.txt
[OK] 合并完成：1_Jon_Gina_talk → 1.txt
[OK] 合并完成：2_John_Maria_talk → 2.txt
[OK] 合并完成：3_Joanna_Nate_talk → 3.txt
[OK] 合并完成：4_Tim_John_talk → 4.txt
[OK] 合并完成：5_Audrey_Andrew_talk → 5.txt
[OK] 合并完成：6_James_John_talk → 6.txt
[OK] 合并完成：7_Deborah_Jolene_talk → 7.txt
[OK] 合并完成：8_Evan_Sam_talk → 8.txt
[OK] 合并完成：9_Calvin_Dave_talk → 9.txt
全部完成！输出位置： /root/nfs/hmj/proj/mem0/evaluation/src/dataset/all_conversations


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache

"""
--------------------------------------
 M/M/1 Queue — Controlled Service Rate μ
 Value Iteration + Policy Iteration
--------------------------------------
State i = number of customers in the system
Action μ ∈ μ_list (discretized service rate)
"""

# ==============================
# 1. 参数设置
# ==============================
lambda_arr = 2.0          # 到达率 λ
beta = 0.95               # 折扣因子 β
nu = 1.0                  # Unif rate used in Bellman
max_state = 50            # 最大状态 i ∈ {0,...,50}
mu_list = np.linspace(0.1, 6.0, 40)   # 离散化服务速率 μ

# 成本函数 ---------------------------------
def c(i):
    """等待成本，可换成线性、二次、指数等"""
    return i                  # 线性
    # return i**2             # 二次
    # return np.exp(0.1*i)    # 指数

def q(mu):
    """服务成本，可改"""
    return 0.5 * mu**2        # 二次服务成本
    # return mu               # 线性
    # return np.exp(0.2*mu)   # 指数

# ==============================
# 2. Bellman 更新（值迭代）
# ==============================
def value_iteration(max_iter=500, tol=1e-6):
    J = np.zeros(max_state+1)
    policy = np.zeros(max_state+1)

    for iteration in range(max_iter):
        J_new = np.zeros(max_state+1)

        # i = 0
        J_new[0] = (c(0) + (nu - lambda_arr) * J[0] + lambda_arr * J[1]) / (beta + nu)

        # i >= 1
        for i in range(1, max_state+1):

            cost_list = []
            for mu in mu_list:
                # Bellman 方程
                v = (
                    c(i) + q(mu)
                    + lambda_arr * J[i+1 if i < max_state else i]
                    + mu * J[i-1]
                    + (nu - lambda_arr - mu) * J[i]
                ) / (beta + nu)
                cost_list.append(v)

            J_new[i] = min(cost_list)
            policy[i] = mu_list[np.argmin(cost_list)]

        if np.max(np.abs(J_new - J)) < tol:
            print(f"Value Iteration Converged at iteration {iteration}")
            break
        J = J_new

    return J, policy


# ==============================
# 3. 策略迭代（Policy Iteration）
# ==============================
def policy_iteration(max_iter=50, tol=1e-6):
    policy = np.ones(max_state+1) * mu_list[0]
    J = np.zeros(max_state+1)

    for k in range(max_iter):

        # ---- Policy Evaluation ----
        while True:
            J_new = np.zeros(max_state+1)
            J_new[0] = (
                c(0) + (nu - lambda_arr) * J[0] + lambda_arr * J[1]
            ) / (beta + nu)

            for i in range(1, max_state+1):
                mu = policy[i]
                J_new[i] = (
                    c(i) + q(mu)
                    + lambda_arr * J[i+1 if i < max_state else i]
                    + mu * J[i-1]
                    + (nu - lambda_arr - mu) * J[i]
                ) / (beta + nu)

            if np.max(np.abs(J_new - J)) < tol:
                break
            J = J_new

        # ---- Policy Improvement ----
        stable = True
        for i in range(1, max_state+1):
            costs = []
            for mu in mu_list:
                v = (
                    c(i) + q(mu)
                    + lambda_arr * J[i+1 if i < max_state else i]
                    + mu * J[i-1]
                    + (nu - lambda_arr - mu) * J[i]
                ) / (beta + nu)
                costs.append(v)

            new_mu = mu_list[np.argmin(costs)]
            if abs(new_mu - policy[i]) > 1e-8:
                policy[i] = new_mu
                stable = False

        if stable:
            print(f"Policy Iteration converged at iteration {k}")
            break

    return J, policy


# ==============================
# 4. 仿真模拟验证策略
# ==============================
def simulate(policy, T=5000):
    i = 0
    total_cost = 0

    for t in range(T):
        mu = policy[i]

        # 状态转移（离散事件模拟）
        r = np.random.rand()

        # 到达
        if r < lambda_arr / (lambda_arr + mu):
            i = min(i+1, max_state)
        else:
            if i > 0:
                i -= 1

        total_cost += c(i) + q(mu)

    return total_cost / T


# ==============================
# 运行
# ==============================
if __name__ == "__main__":

    J_vi, policy_vi = value_iteration()
    J_pi, policy_pi = policy_iteration()

    print("Optimal policy (Value Iteration):")
    print(policy_vi)

    print("\nOptimal policy (Policy Iteration):")
    print(policy_pi)

    # 仿真验证
    avg_cost = simulate(policy_vi)
    print("\nSimulated average discounted cost:", avg_cost)

    # 可视化策略
    plt.plot(policy_vi, label="μ*(i) — Value Iteration")
    plt.xlabel("State i")
    plt.ylabel("Optimal μ")
    plt.legend()
    plt.grid()
    plt.show()


/tmp/ipykernel_2183952/4064754723.py:56: RuntimeWarning: overflow encountered in scalar add
  c(i) + q(mu)
/tmp/ipykernel_2183952/4064754723.py:59: RuntimeWarning: overflow encountered in scalar multiply
  + (nu - lambda_arr - mu) * J[i]
/tmp/ipykernel_2183952/4064754723.py:58: RuntimeWarning: overflow encountered in scalar multiply
  + mu * J[i-1]
/tmp/ipykernel_2183952/4064754723.py:56: RuntimeWarning: invalid value encountered in scalar add
  c(i) + q(mu)
/tmp/ipykernel_2183952/4064754723.py:99: RuntimeWarning: overflow encountered in subtract
  if np.max(np.abs(J_new - J)) < tol:
/tmp/ipykernel_2183952/4064754723.py:87: RuntimeWarning: overflow encountered in scalar add
  c(0) + (nu - lambda_arr) * J[0] + lambda_arr * J[1]
/tmp/ipykernel_2183952/4064754723.py:93: RuntimeWarning: invalid value encountered in scalar add
  c(i) + q(mu)


KeyboardInterrupt: 